### Heroes Of Pymoli Data Analysis  - E.Brown 10/11/2018
CUSTOMER DEMOGRAPHICS
* The dataset represents 576 active players with purchase history.  I assumed that each unique screen name represents a different player. Additional analysis would be needed to confirm this. 
* Male player are the majority of the Heroes of Pymoli customer base (84%), with the next largest player base of females at (14%).  
* Heroes of Pymoli's peak age demographic falls between 20-24 year (44.79%) with secondary groups falling between 15-19 year (18.58%) and 25-29 years (13.37%).
* Note:  Players in lower age groups may have purchasing power influenced by players in the higher age groups.
* On average, men spent $0.18 to $0.33 less per purchase than other gender groups, though data is based on a larger number of purchases.


PURCHASE ANALYIS
* 108 unique items were purchased by active players, with an average price of $3.05 each.
* Oathbreaker, Last Hope of the Breaking Storm, was the most popular item and the most profitable item. 
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import pandas as pd
import numpy as np
#Hide Warning Messsages
import warnings
warnings.filterwarnings('ignore')

In [2]:
# File location
csvfile = "Resources/purchase_data.csv"
# Read Purchasing File and store into Pandas data frame, preview 3 lines
purchase_data = pd.read_csv(csvfile)
purchase_data.head(3)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88


In [3]:
purchase_data.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,92.114103,3.050987
std,225.310896,6.659444,52.775943,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,48.000000,1.980000
50%,389.500000,22.000000,93.000000,3.150000
75%,584.250000,25.000000,139.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


Observations:
The dataframe appears to have repeat buyers. SN is not a primary key.
There are 780 record in this dataset. 

## Player Count

* Display the total number of players


In [4]:
# Return number of unique Screen Names as number of players
total_players = purchase_data['SN'].nunique()
print("Assuming each screen name is a player, there are %s total players in this dataset."% total_players)

Assuming each screen name is a player, there are 576 total players in this dataset.


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
# Run basic calculations on the purchase_data
num_items = purchase_data['Item ID'].nunique()
avg_price = purchase_data['Price'].mean()
num_purchases = purchase_data['Purchase ID'].count()
total_revenue = purchase_data['Price'].sum()
purchase_analysis = pd.DataFrame({"Number of Unique Items": [num_items],
                                  "Average_Price": [avg_price],
                                  "Number of Purchases": [num_purchases],
                                  "Total_Revenue": total_revenue})
purchase_analysis

,Number of Unique Items,Average_Price,Number of Purchases,Total_Revenue
0,183,3.050987,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
# determine the gender demographics of the purchase_data
purchase_data.head(1)
# count and percentage of Unique Screen Names by gender
gender = pd.DataFrame(purchase_data.groupby("Gender")['SN'].nunique())
gender_renamed = gender.rename(columns={"SN": "Total Counts"})
gender_renamed['Percentage of Players'] = gender / total_players * 100

gender_renamed


,Total Counts,Percentage of Players
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# determine the gender demographics of the purchase_data
purchase_data.head(1)
# Select Purchase ID, Price,  SN
purchase_analysis = purchase_data[["SN", "Gender", "Price"]]
# Group By Gender
purchase_analysis_group = purchase_analysis.groupby(["Gender"])

In [8]:
# Calculate averages for purchase data by Gender Only
# Purchase Count
purchase_count = purchase_analysis_group['SN'].count()

#Average Purchase Price
average_purchase_price = purchase_analysis_group['Price'].mean().round(2)

#Total Purchase Value By Gender
total_purchase_value = purchase_analysis_group['Price'].sum()

In [9]:
# Average Total Purchase PER PERSON by gender  ## I HAD A HARD TIME WITH THIS, IS THERE A BETTER WAY?
purchase_analysis_group_person = pd.DataFrame(purchase_analysis.groupby(['Gender','SN']).sum())
purchase_analysis_group_person = purchase_analysis_group_person.groupby(['Gender'])['Price'].mean().round(2)
purchase_analysis_group_person 


Gender
Female                   4.47
Male                     4.07
Other / Non-Disclosed    4.56
Name: Price, dtype: float64

In [10]:
#Return dataframe of purchase analysis by gender
purchase_analysis_gender = pd.DataFrame({"Purchase Count" : purchase_count,
                                         "Average Purchase Price" : average_purchase_price,
                                         "Total Purchase Value" : total_purchase_value,
                                         "Avg Total Purchase per Person" : purchase_analysis_group_person})

#convert column to currency
purchase_analysis_gender["Average Purchase Price"] = purchase_analysis_gender["Average Purchase Price"].map("${:.2f}".format)
purchase_analysis_gender["Total Purchase Value"] = purchase_analysis_gender["Total Purchase Value"].map("${:.2f}".format)
purchase_analysis_gender["Avg Total Purchase per Person"] = purchase_analysis_gender["Avg Total Purchase per Person"].map("${:.2f}".format)
purchase_analysis_gender


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [11]:
#original dataframe
purchase_data.head(2)
#new dataframe with no duplicates
age_data = purchase_data.drop_duplicates('SN')
age_data.count()  # This should equal 576

Purchase ID    576
SN             576
Age            576
Gender         576
Item ID        576
Item Name      576
Price          576
dtype: int64

In [12]:
#Create the age bins
# bins are <10, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40+
bins = [-1,9,14,19,24,29,34,39,101]
#Create names for bins  (one less than bins)
bin_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39","40+"]
age_data["Age Demographics"] = pd.cut(age_data["Age"],bins,labels=bin_names)
purchase_data_age = age_data.groupby("Age Demographics")

In [13]:
#Calculate count and percentage for each bin
age_count = purchase_data_age['Age'].count()
percentage_of_players = age_count / total_players
age_demo = pd.DataFrame({"Age Count" : age_count, "Percentage of Players": percentage_of_players})
age_demo['Percentage of Players'] = age_demo['Percentage of Players'].map("{:,.2%}".format)

age_demo


,Age Count,Percentage of Players
Age Demographics,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [14]:
# use the same bins as were use in Age Demographics
#Calculate coun and averages
purchase_count_age = purchase_data_age['Purchase ID'].count()
avg_purchase_price_age = purchase_data_age['Price'].mean()
total_purchase_value_age = purchase_data_age['Price'].sum()

In [15]:
#NEED TO ADD IN THE AVG TOTAL PRICE PER PERSON
#Use original dataframe (with duplicate SN to get the avg by SN)
#new dataframe with no duplicates
# Average Total Purchase PER SN by age bin
purchase_data["Age Demographics"] = pd.cut(purchase_data["Age"],bins,labels=bin_names)
purchase_analysis_age_person = pd.DataFrame(purchase_data.groupby(['Age Demographics','SN']).sum())
purchase_analysis_age_person = purchase_analysis_age_person.groupby(['Age Demographics'])['Price'].mean().round(2)
purchase_analysis_age_person 

Age Demographics
<10      4.54
10-14    3.76
15-19    3.86
20-24    4.32
25-29    3.81
30-34    4.12
35-39    4.76
40+      3.19
Name: Price, dtype: float64

In [16]:
#Create a new dateframe
age_purchase_analysis = pd.DataFrame({"Purchase Count" : purchase_count_age,
                                      "Average Purchase Price" : avg_purchase_price_age,
                                      "Total Purchase Value" : total_purchase_value_age,
                                     "Avg Total Purchase per Person" : purchase_analysis_age_person})


#format columns
age_purchase_analysis["Average Purchase Price"] = age_purchase_analysis["Average Purchase Price"].map("${:.2f}".format)
age_purchase_analysis["Total Purchase Value"] = age_purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
age_purchase_analysis["Avg Total Purchase per Person"] = age_purchase_analysis["Avg Total Purchase per Person"].map("${:.2f}".format)

age_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Demographics,,,,
<10,17,$3.39,$57.63,$4.54
10-14,22,$3.07,$67.64,$3.76
15-19,107,$3.10,$331.88,$3.86
20-24,258,$3.06,$790.39,$4.32
25-29,77,$2.91,$223.93,$3.81
30-34,52,$2.92,$151.92,$4.12
35-39,31,$3.51,$108.81,$4.76
40+,12,$3.04,$36.45,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [17]:
# Group by unique SN, but no bins

purchase_data.head(2)
# Group By SN
purchase_data_SN = purchase_data.groupby(["SN"])
# Calculations
purchase_count_SN = purchase_data_SN['Purchase ID'].count()
avg_purchase_price_SN = purchase_data_SN['Price'].mean()
total_purchase_value_SN = purchase_data_SN['Price'].sum()
#create new dataframe
SN_purchase_analysis = pd.DataFrame({"Purchase Count" : purchase_count_SN,
                                      "Average Purchase Price" : avg_purchase_price_SN,
                                      "Total Purchase Value" : total_purchase_value_SN})
SN_purhase_analysis = SN_purchase_analysis.sort_values(['Total Purchase Value'], ascending = False, inplace = True)
#format columns
SN_purchase_analysis["Average Purchase Price"] = SN_purchase_analysis["Average Purchase Price"].map("${:.2f}".format)
SN_purchase_analysis["Total Purchase Value"] = SN_purchase_analysis["Total Purchase Value"].map("${:.2f}".format)
SN_purchase_analysis.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [18]:
#Create df using the original purchase data information, extract Item ID, Item Name, and Price
popular_items = purchase_data[['Item ID','Item Name','Price']]
#Groupby Item ID and Item Name
popular_items_group = popular_items.groupby(['Item ID', 'Item Name'])
# Calculations
purchase_count_pop = popular_items_group['Item ID'].count()
item_price_pop = popular_items_group['Price'].mean()
total_purchase_value_pop = popular_items_group['Price'].sum()
#create new dataframe
pop_item_analysis = pd.DataFrame({'Purchase Count' : purchase_count_pop,
                                  'Item Price' : item_price_pop,
                                  'Total Purchase Value' : total_purchase_value_pop})
pop_item_analysis_sorted = pop_item_analysis.sort_values(['Purchase Count'], ascending = False)
#format columns
pop_item_analysis_sorted["Item Price"] = pop_item_analysis_sorted["Item Price"].map("${:.2f}".format)
pop_item_analysis_sorted["Total Purchase Value"] = pop_item_analysis_sorted["Total Purchase Value"].map("${:.2f}".format)
pop_item_analysis_sorted.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [19]:
#sort the popular item dataframe by Total purchase value, descending. Don't use the reformatted version.
profit_item_analysis = pop_item_analysis.sort_values(['Total Purchase Value'], ascending = False)
#format columns
profit_item_analysis["Item Price"] = profit_item_analysis["Item Price"].map("${:.2f}".format)
profit_item_analysis["Total Purchase Value"] = profit_item_analysis["Total Purchase Value"].map("${:.2f}".format)

profit_item_analysis.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
